# P3 Avistamiento de aves

## Obtener datos (scraping)

Primero vamos a obtener los datos. Para ello nos visitaremos las web de avistamientos de aves [shorebirder](https://www.shorebirder.com/), [trevorsbirding](https://www.trevorsbirding.com/) y [dantallmansbirdblog](https://dantallmansbirdblog.blogspot.com/).

Durante la visita a la web y haciendo uso del inspector (F12) podemos ver que las descripciones que necesitamos se encuentran en los tag de párrafo (entre *\<p\> TEXTO \</p\>*). Sabiendo eso vamos a crear funciones de utilidad que se encargarán de descargar el contenido de la web y extraer el texto.

Las descargas las realizaremos en `data/raw` mientras que en `data/posts` guardaremos los textos encontrados.

** `dantallmansbirdblog` tiene una estructura ligeramente diferente (entre *\<p\>\</p\> TEXTO \<p\>\</p\>*), a lo que tendremos que modificar la función `get_texts` (a continuación) para obtener sus textos.

In [1]:
# I/O utils
import os
from os.path import exists
import re
import wget
import tqdm
import json

# auxiliar regex
remove_parentesis_text = re.compile(r'\(.*\)')
sentence_endings = re.compile(r'\.|\*')

# auxiliar functions
def maybe_mkdir(path):
  try:
    if not exists(path):
      os.mkdir(path)
  except OSError as error:
    print(error)

def download(url, out_label):
  filepath = f"{data_raw_path}/{out_label}"
  if exists(filepath):
    os.remove(filepath)
  return wget.download(url, out=filepath)

def get_texts(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()

  # get texts
  get_p = re.compile(r'<p>((.|\n)*?)</p>')
  texts = get_p.findall(text)

  # remove styling and inner tags
  remove_tags = re.compile(r'(<.*?>)|\\n| +(?= )|\\|\&.+?\;')
  return map(lambda text: re.sub(remove_tags, "", str(text[0]).lower()), texts)

def write(path, filename, data):
  filepath = f"{path}/{filename}.txt"
  file = open(filepath, "w", encoding="utf-8")
  for item in data:
    file.write(str(item)+"\n")
  file.close()
  return filepath

def save_json(path, filename, dict):
  filepath = f"{path}/{filename}.json"
  # create json object from dictionary
  parsed_json = json.dumps(dict)
  f = open(filepath,"w")
  # write json object to file
  f.write(parsed_json)
  # close file
  f.close()
  return filepath

# constants
data_posts_cache = "../data/cache" # guardar resultados de queries a sparql
data_raw_path = "../data/raw" # descargas
data_posts_path = "../data/posts" # guardar los textos de los post scrapeados
data_results_path = "../data/results" # guardar los resultados de las diferentes pruebas

# build directory structure
maybe_mkdir("../models")
maybe_mkdir("../models/entity_ruler")
maybe_mkdir("../data")
maybe_mkdir("../owl")
maybe_mkdir(data_raw_path)
maybe_mkdir(data_posts_path)
maybe_mkdir(data_results_path)

A continuación haciendo uso de las funciones anteriores scrapeamos la home de `shorebirder`.

In [2]:
# scrap shorebirder.com
shorebirder_filename = "shorebirder_home.html"
shorebirder_home = download("https://www.shorebirder.com/", shorebirder_filename)
posts = get_texts(shorebirder_home)
shorebirder_posts_file = write(data_posts_path, shorebirder_filename, posts)

open(shorebirder_posts_file, "r").readlines()[0]

"my late march solo visit to norway is in the books and was about as much fun as i've had in a while. the middle few days of the trip were spent birding around varanger, bookended by more touristy time intromsÃ£Â¸and oslo. at some point in the coming months there will be a full trip report here plus a very detailed cloudbirders submission. in the meantime, here is some proof that i actually went.\n"

Lo mismo para `trevorsbirding`.

In [3]:
# scrap trevorsbirding.com
trevorsbirding_filename = "trevorsbirding_home.html"
trevorsbirding_home = download("https://www.trevorsbirding.com/", trevorsbirding_filename)
posts = get_texts(trevorsbirding_home)
trevorsbirding_posts_file = write(data_posts_path, trevorsbirding_filename, posts)

open(trevorsbirding_posts_file, "r").readlines()[2]

'earlier this week i glanced out of my sunroom window to check whether there were any birds at my birdbaths. i currently have three birdbaths just outside the room, one on the ground, one on a pedestal at about 60cm and one hanging from a tree branch at a height of about 1.5 metres. i was delighted to see a small flock of purple-crowned lorikeets having a drink and dipping into the water for a bath. i have just checked my list of species to have visited the birdbaths. this was bird species number 36, in addition to the three reptiles and two mammal species.\n'

## Intento 1: Usar spacy sin modificar

In [4]:
# Cargar dependencias para el nlp
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_lg")

In [5]:
# Prueba
for text in open(shorebirder_posts_file, "r").readlines()[3:5]:
  train = nlp(text)
  displacy.render(train, jupyter=True, style="ent")

Vemos que es capaz de identificar diferentes entidades dentro de las frases, pero no pájaros.

## Intento 2: Usando sparql query para encontrar aves

Idea: Usando el tokenizer de spacy como tokenizer trocear las frases. A partir de los token etiquetados como nombre (`NOUN`) lanzamos una petición a la dbpedia. Ya el resultado de la dbpedia nos dirá si existe y cual es su etiqueta / url.

In [6]:
# SparQL class extension
# Prefixes and Class based from https://github.com/ejrav/pydbpedia
from SPARQLWrapper import SPARQLWrapper, JSON

class SparqlEndpoint(object):

    def __init__(self, endpoint, prefixes={}):
        self.sparql = SPARQLWrapper(endpoint)
        self.prefixes = {
            "dbo": "http://dbpedia.org/ontology/",
            "owl": "http://www.w3.org/2002/07/owl#",
            "xsd": "http://www.w3.org/2001/XMLSchema#",
            "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
            "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dc": "http://purl.org/dc/elements/1.1/",
            "dbpedia2": "http://dbpedia.org/property/",
            "dbpedia": "http://dbpedia.org/",
            "skos": "http://www.w3.org/2004/02/skos/core#",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "yago": "http://dbpedia.org/class/yago/",
            }
        self.prefixes.update(prefixes)
        self.sparql.setReturnFormat(JSON)

    def query(self, q):
        lines = ["PREFIX %s: <%s>" % (k, r) for k, r in self.prefixes.items()]
        lines.extend(q.split("\n"))
        query = "\n".join(lines)
        self.sparql.setQuery(query)
        results = self.sparql.query().convert()
        return results["results"]["bindings"]


class DBpediaEndpoint(SparqlEndpoint):
    def __init__(self, endpoint, prefixes = {}):
        super(DBpediaEndpoint, self).__init__(endpoint, prefixes)

s = DBpediaEndpoint(endpoint = "http://dbpedia.org/sparql")

In [7]:
# Función para buscar una ave dado su nombre
def search_bird_dbpedia(token):
  return s.query('''
    SELECT *
    WHERE {
      ?bird a dbo:Bird ;
            rdfs:label ?name ;
            dbo:abstract ?comment .

      filter (!isLiteral(?name) ||
              langmatches(lang(?name), "en")) .

      filter (!isLiteral(?comment) ||
              langmatches(lang(?comment), "en")) .

      filter (CONTAINS(LCASE(STR(?name)), "{token}")) .
    }
    limit 5
  '''.replace("{token}", token))

search_bird_dbpedia("falcon")

[{'bird': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Collared_falconet'},
  'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Collared falconet'},
  'comment': {'type': 'literal',
   'xml:lang': 'en',
   'value': 'The collared falconet (Microhierax caerulescens) is a species of bird of prey in the family Falconidae. It is found in the Indian Subcontinent and Southeast Asia, ranging across Bangladesh, Bhutan, Cambodia, India, Laos, Myanmar, Nepal, Thailand, Malaysia, and Vietnam.Its natural habitat is temperate forest, often on the edges of broadleaf forest. It is 18 cm long. Rapid wingbeats are interspersed with long glides. When perched, it is described as being "rather shrikelike."'}},
 {'bird': {'type': 'uri',
   'value': 'http://dbpedia.org/resource/Peregrine_falcon'},
  'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Peregrine falcon'},
  'comment': {'type': 'literal',
   'xml:lang': 'en',
   'value': 'The peregrine falcon (Falco peregrinus), also known

In [8]:
# Prueba
nlp = spacy.load("en_core_web_lg")
maybe_matches = {}
for text in open(shorebirder_posts_file, "r").readlines()[4:5]:
  doc = nlp(text)
  for chunk in doc.noun_chunks:
    for token in chunk:
      if token.pos_ == 'NOUN':
        results = search_bird_dbpedia(token.lemma_)
        if len(results) > 0:
          maybe_matches[token] = results
          print(token)

body
time
ol'
car
gull
ring
gull
wing
peregrine
falcon


Vale, funciona? Lo que es muy lento y estamos machacando la dbpedia a queries.

## Intento 3: Cachear / indexar la dbpedia

Del intento anterior vamos a coger los resultados de todos los pájaros y lo convertiremos en un diccionario para que nos sea más fácil buscar y solo haremos n queries a la dbpedia. Por supuesto, esta estrategia es solo factible si el conjunto es finito. Como es nuestro caso, va haber n especies de pájaros, pero no va a estar creciendo dia a dia.

### Estrategia
- Obtener lista de todos los nombres de pájaros.
- Con spacy analizaremos la entrada del avistamiento y obtenemos los `noun chunk`.
  - Para hacer spacy más rápido vamos a deshabilitar las pipelines que no usemos, que son `lemmatizer` y `ner`.
- Con cada `chunk` usando fuzzy-search en la lista de nombres de pájaros para encontrar aquellos chunk que parezcan nombres de pájaros.

In [9]:
# Obtener todos los pájaros. Obtenemos las descripciones para intentos posteriores.
birds_sparql = s.query("""
  SELECT DISTINCT *
  WHERE {
    ?bird a dbo:Bird ;
          rdfs:label ?name ;
          dbo:abstract ?comment .

    filter (!isLiteral(?name) ||
            langmatches(lang(?name), "en")) .

    filter (!isLiteral(?comment) ||
            langmatches(lang(?comment), "en")) .
    
  }
  limit 10000
""")

birds_sparql += s.query("""
  SELECT DISTINCT *
  WHERE {
    ?bird a dbo:Bird ;
          rdfs:label ?name ;
          dbo:abstract ?comment .

    filter (!isLiteral(?name) ||
            langmatches(lang(?name), "en")) .

    filter (!isLiteral(?comment) ||
            langmatches(lang(?comment), "en")) .
    
  }
  limit 10000
  offset 10000
""")

write("../data", "birds", birds_sparql)

print(f"Hemos obtenido los nombres de {len(birds_sparql)} pájaros")
for d in birds_sparql[0:5]:
  print(d['name']['value'])

Hemos obtenido los nombres de 10369 pájaros
Actenoides
African goshawk
African pitta
African red-eyed bulbul
Alcedo


Convertimos los datos en crudo a un diccionario

In [10]:
# sparql a diccionario
remove_parentesis_text = re.compile(r'\(|\)')

birds = {}
for bird in birds_sparql:
  bird_name_lower = bird["name"]["value"].lower()
  key = re.sub(remove_parentesis_text, "", str(bird_name_lower))
  if key in birds.keys():
    print(f"bird {key} is duplicated.", birds[key]["name"], bird["name"]["value"])
  birds[key] = {
    "name": bird["name"]["value"],
    "url": bird["bird"]["value"],
    "description": bird["comment"]["value"],
  }
bird_keys = birds.keys() # buscaremos por las key
assert len(birds_sparql) == len(bird_keys) # aseguramos que no hayamos perdido ninguna key

In [11]:
birds["goliath mangalia"] # bueno... la dbpedia tampoco es perfecta

{'name': 'Goliath (Mangalia)',
 'url': 'http://dbpedia.org/resource/Goliath_(Mangalia)',
 'description': 'Goliath is the name of a crane that is currently located at the Mangalia shipyard in Mangalia, Romania. Formerly, it was part of the Fore River Shipyard in Quincy, Massachusetts.'}

In [25]:
nlp = spacy.load("en_core_web_lg", disable=['lemmatizer'])

doc = nlp("Black-billed flycatcher")
displacy.render(doc, jupyter=True)


Podemos ver que el `tokenizer` de spacy nos separa las palabras compuestas con guion.

Para solucionarlo vamos a modificar el tokenizer para que no separe las palabras con guion.

In [26]:
# hacer que el tokenizer no separe palabras con guion
# https://stackoverflow.com/questions/59993683/how-can-i-get-spacy-to-stop-splitting-both-hyphenated-numbers-and-words-into-sep

from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

def custom_tokenizer(nlp):
    inf = list(nlp.Defaults.infixes)               # Default infixes
    inf.remove(r"(?<=[0-9])[+\-\*^](?=[0-9-])")    # Remove the generic op between numbers or between a number and a -
    inf = tuple(inf)                               # Convert inf to tuple
    infixes = inf + tuple([r"(?<=[0-9])[+*^](?=[0-9-])", r"(?<=[0-9])-(?=-)"])  # Add the removed rule after subtracting (?<=[0-9])-(?=[0-9]) pattern
    infixes = [x for x in infixes if '-|–|—|--|---|——|~' not in x] # Remove - between letters rule
    infix_re = compile_infix_regex(infixes)

    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)

In [27]:
# Test
nlp.tokenizer = custom_tokenizer(nlp)

doc = nlp("Black-billed flycatcher")
displacy.render(doc, jupyter=True)

Genial! A demás conseguimos que `Black-billed` se detecte como adjetivo y no como adjetivo + verbo. 

In [28]:
# Busqueda de pájaros por fuzzysearch
from fuzzywuzzy import fuzz
def search_bird_dict(chunk):
  best_match = (False, None, 0)
  for key in bird_keys:
    score = fuzz.ratio(key, chunk)
    if score > 80 and score > best_match[-1]:
      # print(f"\tchunk: '{chunk}' compare '{key}' score: '{score}'")
      best_match = (True, key, score)
      break
  return best_match

def get_nouns(chunk):
  only_nouns = []
  for token in chunk:
    if token.pos_ == "NOUN" or token.pos_ == "ADJ":
      only_nouns.append(token.lower_)

  if len(only_nouns) > 0:
    return " ".join(only_nouns)
  return None

maybe_matches = []
for text in open(shorebirder_posts_file, "r").readlines()[0:5]:
  doc = nlp(text)
  for chunk in [get_nouns(chunk) for chunk in doc.noun_chunks]:
    if chunk != None:
      (found, bird_key, score) = search_bird_dict(chunk)
      if found:
        maybe_matches.append(bird_key)
        print(f"Found '{bird_key}' in '{chunk}' with score of {score}")
          
# Test result
print(maybe_matches)

Found 'ring-billed gull' in 'ring-billed gull' with score of 100
Found 'peregrine falcon' in 'female peregrine falcon' with score of 82
['ring-billed gull', 'peregrine falcon']


In [54]:
# preparamos la ontología
# pip install Cython owlready2
# pip install slugify
import Cython
from owlready2 import *
from slugify import slugify
import uuid

# Creamos una ontología
onto = get_ontology("http://avistamiento-aves.org/onto.owl")
onto.destroy()
onto = get_ontology("http://avistamiento-aves.org/onto.owl")

with onto:
  class label(DataProperty):
    range = [str]
  class match_text(DataProperty):
    range = [str]
  class match_score(DataProperty):
    range = [float]
  class dbpedia(DataProperty):
    range = [str]
  class date(DataProperty):
    range = [str]
  class position(DataProperty):
    range = [str]
  class generic_context(DataProperty):
    range = [str]

  class Bird(Thing):
    pass
  class Sighting(Thing):
    pass

  class has_been_seen(ObjectProperty):
    domain = [Bird]
    range = [Sighting]
  class birds_found(ObjectProperty):
    domain = [Sighting]
    range = [Bird]
    inverse_property = has_been_seen

# Owl functions
def create_sighting(doc):
  sighting = Sighting(f"sighting-{uuid.uuid4()}")
  sighting.text = doc.text
  return sighting

def register_bird(sighting, bird_key, chunk_nouns, score):
  slugify_name = slugify(bird_key, separator="_")
  bird_onto = types.new_class(slugify_name, (Bird,))
  bird_dict = birds[bird_key]
  bird_individual = bird_onto(f"{slugify_name}-{uuid.uuid4()}")
  bird_individual.label.append(bird_dict["name"])
  bird_individual.match_text.append(chunk_nouns)
  bird_individual.match_score.append(score)
  bird_individual.dbpedia.append(bird_dict["url"])
  sighting.birds_found.append(bird_individual)
  return bird_individual

def add_context(individual, name, type):
  if type == "DATE":
    individual.date.append(name)
  if type == "NORP" or type == "GPE":
    individual.position.append(name)
  else:
    individual.generic_context.append(name)

# avistamiento
class sighting_class:
  namespace = onto

  def __init__(self, doc):
    self.doc = doc
    self.birds = []
    self.contexts = []

  def add_bird(self, bird):
    self.birds.append(bird)

  def add_context(self, context):
    self.contexts.append(context)

class bird_class:
  def __init__(self, bird_key, text_found, similarity):
    self.bird_key = bird_key
    self.text_found = text_found
    self.similarity = similarity
  def __repr__(self):
        return self.bird_key

# https://stackoverflow.com/questions/50792574/what-are-the-supported-date-and-time-formats-in-spacy-2-0/50794950#50794950
class context_class:
  def __init__(self, ent):
    self.ent = ent
    self.name = str(ent)
    self.type = ent.label_


In [55]:
# calcular para todas las review
trace = []
for text in tqdm.tqdm(open(shorebirder_posts_file, "r").readlines()):
  doc = nlp(text)
  sighting = create_sighting(doc)

  for chunk in doc.noun_chunks:
    chunk_nouns = get_nouns(chunk)
    if chunk_nouns != None:
      (found, bird_key, score) = search_bird_dict(chunk_nouns)
      if found:
        bird_individual = register_bird(sighting, bird_key, chunk_nouns, score)
        trace.append(f"Found '{bird_key}' in '{chunk_nouns}' with score of {score}")
        for ent in chunk.ents:
          add_context(bird_individual, str(ent), ent.label_)
      else:
        for ent in chunk.ents:
          add_context(sighting, str(ent), ent.label_)

onto.save(file="../owl/avistamiento-aves.xml", format = "rdfxml")
log_result = write(data_posts_path, "shorebirder_results_log_3", trace)
print(log_result)
open(log_result, "r", encoding="utf-8").readlines()[0:2]

100%|██████████| 56/56 [00:29<00:00,  1.92it/s]

../data/posts/shorebirder_results_log_3.txt


["Found 'ring-billed gull' in 'ring-billed gull' with score of 100\n",
 "Found 'peregrine falcon' in 'female peregrine falcon' with score of 82\n"]

Primera versión funcional sin machacar a la dbpedia. Pero sigue siendo muy lento. Vamos a seguir probando.

## 4. Nueva pipeline [entity_ruler](https://spacy.io/api/entityruler)

En esta aproximación vamos a añadir una pipe más al nlp `en_core_web_lg` pre-entrenado de spacy. Para ello necesitamos hacer una lista de todos los patterns que queramos poner. Es decir, debemos introducir los nombres de los pájaros que queremos que se detecten como patterns y añadir la nueva pipe al nlp.

In [20]:
# usando entity_ruler https://spacy.io/usage/rule-based-matching#entityruler
from spacy.lang.en import English

tag = "BIRD"

# init blank nlp
nlp = English()
nlp.tokenizer = custom_tokenizer(nlp)

# Añadir los nombres de pájaros
patterns = []
for key in bird_keys:
  bird = birds[key]
  doc = nlp(bird["name"])
  pattern = []
  for token in doc:
    pattern.append({
      "LOWER": token.lower_
    })

  patterns.append({
    "label": tag,
    "pattern": pattern,
    "id": key
  })

# add entity_ruler
ruler = nlp.add_pipe("entity_ruler")
with nlp.select_pipes(enable="tagger"):
  ruler.add_patterns(patterns)

nlp.to_disk("../models/entity_ruler/")

In [21]:
doc = nlp("that feeding gulls flock continued to produce by sucking in passers by. at one point a Bonaparte's gull got in on the action, and a flock of 21 common terns appeared from the east and eventually settled into that flock.")
displacy.render(doc, jupyter=True, style="ent")
print(nlp.pipe_names)

['entity_ruler']


In [23]:
# calcular para todas las review
maybe_matches = []
for text in tqdm.tqdm(open(shorebirder_posts_file, "r").readlines()):
  doc = nlp(text)
  for ent in doc.ents:
    if ent.label_ == "BIRD" and ent.ent_id_ != "":
      maybe_matches.append(ent.ent_id_)

print(maybe_matches)

100%|██████████| 56/56 [00:00<00:00, 63.96it/s]

['gull', 'ring-billed gull', 'peregrine falcon', 'gull', 'gull', 'gull', 'iceland gull', 'american bittern', 'hawk', 'little blue heron', 'glossy ibis', 'yellow-bellied sapsucker', 'scarlet tanager', 'gull', "bonaparte's gull", 'least sandpiper', 'saltmarsh sparrow', "nelson's sparrow", 'marsh wren', 'american bittern', 'puna plover', 'hummingbird', 'collared inca', 'shining sunbeam', 'chestnut-crested cotinga', 'cock-of-the-rock', 'solitary eagle', 'lyre-tailed nightjar', 'andean cock-of-the-rock', 'wire-crested thorntail', 'hummingbird', 'white-throated toucan', 'green honeycreeper', 'yellow-bellied dacnis', 'owl', 'nighthawk', 'parrot', 'hoatzin', 'elaenia', 'seabird', 'cinclodes', 'purple sandpiper', 'humboldt penguin']


In [21]:
# Pintar y guardar resultado
result_lines = []
for bird_key in set(maybe_matches):
  bird = dict(birds[bird_key])
  name = bird["name"]
  url = bird["url"]
  result_lines.append(f"Hemos encontrado '{name}' con entrada en la dbpedia '{url}'.")

result_lines_file = write(data_posts_path, "shorebirder_results_4", result_lines)
print(f"Hemos encontrado {len(result_lines)} pájaros distintos")
open(result_lines_file, "r").readlines()[0:5]

Hemos encontrado 37 pájaros distintos


["Hemos encontrado 'Least sandpiper' con entrada en la dbpedia 'http://dbpedia.org/resource/Least_sandpiper'.\n",
 "Hemos encontrado 'Chestnut-crested cotinga' con entrada en la dbpedia 'http://dbpedia.org/resource/Chestnut-crested_cotinga'.\n",
 "Hemos encontrado 'Scarlet tanager' con entrada en la dbpedia 'http://dbpedia.org/resource/Scarlet_tanager'.\n",
 "Hemos encontrado 'Humboldt penguin' con entrada en la dbpedia 'http://dbpedia.org/resource/Humboldt_penguin'.\n",
 "Hemos encontrado 'Saltmarsh sparrow' con entrada en la dbpedia 'http://dbpedia.org/resource/Saltmarsh_sparrow'.\n"]

Mejoramos mucho la velocidad por una cantidad aceptable de reconocimiento de aves. Pero no nos detecta todas las que conseguimos detectar en el `intento 3`. Esto es porque la capa de entity_ruler no es parte del modelo sino pattern matching. por ello no es capaz de encontrar las formas en plural de las aves que el intento 3 si encuentra.

## 5. Entrenar ner Spacy

En este apartado vamos a centrarnos en mejorar la detección de spacy entrenando un modelo de clasificación. Las frases de entreno las cogeremos de las descripciones de la dbpedia. Los datos de entreno deberán estar compuestos de una frase más las posiciones de los nombres de pájaros. Para reconocer los pájaros que aparecen en las descripciones usaremos el `entity ruler` del apartado anterior. Al usar el entity ruler para esta tarea vamos a suponer que en la dbpedia no hay faltas de ortografía en las descripciones para que los pájaros sean identificables, y su nombre aparezca al menos una vez.

La [pipeline](https://spacy.io/usage/spacy-101#pipelines) de spacy que se encarga de etiquetar las entities se llama [Entity Recognizer](https://spacy.io/api/entityrecognizer) o `ner`. Según la documentación de spacy sobre como [entrenar](https://spacy.io/usage/training) un modelo, primero necesitamos las frases de entreno correctamente etiquetadas, vectores por palabra a etiquetar y un fichero de configuración con las pipelines a entrenar.

La generación de frases de entreno usaremos el 100% de frases para train y 20% para test (subconjunto de train). He decidido "correr el riesgo" de over-fitting del modelo a la de no registrar pájaros al modelo. He tomado esta decisión dada las escasas frases/pájaro que podemos obtener de la dbpedia.

Para que ner pueda reconocer y comparar las aves vamos a tener que generar [vectores](https://spacy.io/usage/linguistic-features#vectors-similarity). Sabemos que el nlp de spacy no genera vectores para palabras fuera del modelo, o en el caso de `en_core_web_sm` directamente no tiene. Para ello, la generación de vectores se va a realizar mediante la librería [gensim](https://radimrehurek.com/gensim/) y la técnica [Word2Vec](https://es.wikipedia.org/wiki/Word2vec) como sugieren en la documentación de spacy.

El fichero de configuración guardado en `src/birds_config.cfg` se ha generado usando la [herramienta web](https://spacy.io/usage/training#quickstart) que proporciona spacy seleccionando `ner` + `cpu` + `efficiency`. Ner, porque es la pipeline que queremos entrenar. Cpu, porque he tenido problemas para compilar pytorch para usar la gpu y es una opción más compatible. Efficiency porque nosotros le proporcionaremos los vectores.

In [87]:
# Transformar las descripciones en datos de entreno y test para spacy
import gensim
import random
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# init the "base nlp"
nlp = spacy.load("en_core_web_sm")
# add custom tokenizer
nlp.tokenizer = custom_tokenizer(nlp)
# add entity_ruler 
entity_ruler = spacy.load("../models/entity_ruler")
nlp.add_pipe("entity_ruler", source=entity_ruler, before="ner")

def get_vector(ent):
  if ent.has_vector:
    return ent.vector
  else:
    return gensim.models.Word2Vec(ent, min_count = 1,size = 200, window = 6, sg=0)

# build training sentences
training_data = [
  # ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")], tag/label, vector), # example
]
for key in tqdm.tqdm(bird_keys):
  bird = birds[key]
  bird_name = bird["name"]

  if len(bird_name) == 0:
    continue
  
  bird_name = re.sub(remove_parentesis_text, "", str(bird_name))
  description = bird["description"]

  for train_sentence in re.split(sentence_endings, description):
    # usamos el reconocimiento de aves anterior para encontrar las aves
    # suponemos que la dbpedia tiene las aves bien identificadas
    doc = nlp(train_sentence)
    positions = []
    for ent in doc.ents:
      if ent.label_ == tag and ent.ent_id_ != "":
        vector = get_vector(ent)
        positions.append((ent.start_char, ent.end_char, ent.ent_id_, vector))

    if len(positions) > 0:
      training_data.append(
        (train_sentence, positions)
      )

def outer_join(lst1, lst2):
  lst2_names = [name for name, annotations in lst2]
  lst3 = [(name, annotations) for name, annotations in lst1 if not name in lst2_names]
  return lst3

test_data = random.sample(training_data, k=round(len(training_data)*0.2))

# si quitamos las frases de test del modelo de entreno luego no vamos a poder detectar esas aves... no es ideal pero tampoco podemos asegurar que tengamos frases para todos los pájaros
training_data = outer_join(training_data, test_data) 

for text, annotations in training_data[0:5]:
  print(text, [(a, b, c) for a, b, c, d in annotations])
  print("-"*10)

100%|██████████| 15/15 [00:00<00:00, 18.56it/s]

 The type species is Hombron's kingfisher (Actenoides hombroni) [(21, 41, "hombron's kingfisher"), (43, 53, 'actenoides')]
----------
 A molecular study published in 2017 found that the genus Actenoides, as currently defined, is paraphyletic [(58, 68, 'actenoides')]
----------
 The glittering kingfisher in the monotypic genus Caridonax is a member of the clade containing the species in the genus Actenoides [(5, 26, 'glittering kingfisher'), (121, 131, 'actenoides')]
----------
 Green-backed kingfisher (Actenoides monachus) 
 [(1, 24, 'green-backed kingfisher'), (26, 36, 'actenoides')]
----------
 Black-headed kingfisher (Actenoides monachus capucinus) 
 [(1, 24, 'black-headed kingfisher'), (26, 36, 'actenoides')]
----------


In [88]:
# build train set
from spacy.tokens import DocBin
from spacy.util import filter_spans

nlp = English()
# no separar por "-"
nlp.tokenizer = custom_tokenizer(nlp)
# añadir entity_ruler del apartado anterior
entity_ruler = spacy.load("../models/entity_ruler")
nlp.add_pipe("entity_ruler", source=entity_ruler)

def build_db(nlp, data):
  skips = 0
  # the DocBin will store the example documents
  db = DocBin()
  for text, annotations in tqdm.tqdm(data):
    doc = nlp(text)
    ents = []
    for start, end, label, vector in annotations:
      span = doc.char_span(start, end, label=label, vector=vector)
      if span is None:
        skips += 1
      else:
        ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    db.add(doc)
  return (db, skips)

(db, skips) = build_db(nlp, training_data)
print(f"Skipped {skips} entries")
db.to_disk("./birds_train.spacy")

100%|██████████| 26/26 [00:00<00:00, 185.72it/s]

Skipped 0 entries


In [89]:
# build test set
(db, skips) = build_db(nlp, test_data)
print(f"Skipped {skips} entries")
db.to_disk("./birds_test.spacy")

100%|██████████| 6/6 [00:00<00:00, 136.38it/s]

Skipped 0 entries


In [ ]:
# Train the model https://spacy.io/usage/training
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# train the model
!python -m spacy init fill-config birds_config.cfg config.cfg
!python -m spacy train config.cfg --output ../models/custom_ner --paths.train ./birds_train.spacy --paths.dev ./birds_test.spacy

## powershell: (launch from console to use all cpu cores)
# $env:KMP_DUPLICATE_LIB_OK = "True"
# python -m spacy init fill-config birds_config.cfg config.cfg
# python -m spacy train config.cfg --output ../models/custom_ner --paths.train ./birds_train.spacy --paths.dev ./birds_test.spacy


In [17]:
# juntamos el entreno previo con las patterns
nlp = spacy.load("../models/custom_ner/model-best")
nlp.tokenizer = custom_tokenizer(nlp)

entity_ruler = spacy.load("../models/entity_ruler")
nlp.add_pipe("entity_ruler", source=entity_ruler, before="ner")

# ner = spacy.load("en_core_web_sm")
# nlp.add_pipe("ner", name="base_ner", source=ner, after="ner")

# comprobamos las pipes
print(nlp.pipe_names)

# try
doc = nlp("The golden-fronteds fulvetta (Schoeniparus variegaticeps), also known as the gold-fronted fulvetta, is a species of bird in the family Pellorneidae")
displacy.render(doc, jupyter=True, style="ent")

doc = nlp("The African goshawk (Accipiter tachiro) is a species of African bird of prey in the genus Accipiter which is the type genus of the family Accipitridae.")
displacy.render(doc, jupyter=True, style="ent")

doc = nlp("The type species is Hombron's kingfisher (Actenoides hombroni)")
displacy.render(doc, jupyter=True, style="ent")

['tok2vec', 'entity_ruler', 'ner']


In [22]:
# calcular para todas las review
found_with_ruler = 0
found_with_ner = 0
maybe_matches = []
for text in tqdm.tqdm(open(shorebirder_posts_file, "r").readlines()):
  doc = nlp(text)
  # if len(doc.ents) > 0:
  #   displacy.render(doc, jupyter=True, style="ent")
  for ent in doc.ents:
    if ent.label_ == "BIRD":
      found_with_ruler += 1
      maybe_matches.append((ent.ent_id_, str(ent)))
    else:
      found_with_ner += 1
      maybe_matches.append((ent.label_, str(ent)))
     
invalid_birds = 0
# Pintar y guardar resultado
result_lines = []
for bird_key, original in set(maybe_matches):
  bird = dict(birds[bird_key])
  name = bird["name"]
  url = bird["url"]
  (valid, alt_key, accuracy) = search_bird_dict(original)
  if valid:
    result_lines.append(f"Hemos encontrado '{original}' con entrada en la dbpedia [{name}]'{url}'.")
  else:
    invalid_birds += 1
    print(f"valid={valid}, accuracy={accuracy}, alt='{alt_key}' Hemos encontrado '{original}' con entrada en la dbpedia [{name}]'{url}'.")

result_lines_file = write(data_posts_path, "shorebirder_results_5", result_lines)
print(f"Hemos encontrado {len(result_lines)} pájaros distintos de {found_with_ner+found_with_ruler-invalid_birds}.",
  f"{found_with_ner} encontrados con ner y {found_with_ruler} encontrados con entity ruler.",
  f"Ha habido {invalid_birds} falsos casos")
open(result_lines_file, "r", encoding="utf-8").readlines()[0:5]

100%|██████████| 56/56 [00:11<00:00,  4.95it/s]


valid=False, accuracy=0, alt='None' Hemos encontrado 'bridgeport' con entrada en la dbpedia [Booby]'http://dbpedia.org/resource/Booby'.
valid=False, accuracy=0, alt='None' Hemos encontrado 'isla pucusana, home to' con entrada en la dbpedia [African black swift]'http://dbpedia.org/resource/African_black_swift'.
valid=False, accuracy=0, alt='None' Hemos encontrado 'fun event' con entrada en la dbpedia [Red grouse]'http://dbpedia.org/resource/Red_grouse'.
valid=False, accuracy=0, alt='None' Hemos encontrado 'aware' con entrada en la dbpedia [Hummingbird]'http://dbpedia.org/resource/Hummingbird'.
valid=False, accuracy=0, alt='None' Hemos encontrado 'increasing east winds' con entrada en la dbpedia [Sickle-billed vanga]'http://dbpedia.org/resource/Sickle-billed_vanga'.
valid=False, accuracy=0, alt='None' Hemos encontrado 'boat ride' con entrada en la dbpedia [Spotted tanager]'http://dbpedia.org/resource/Spotted_tanager'.
Hemos encontrado 37 pájaros distintos de 43. 6 encontrados con ner y 4

["Hemos encontrado 'ring-billed gull' con entrada en la dbpedia [Ring-billed gull]'http://dbpedia.org/resource/Ring-billed_gull'.\n",
 "Hemos encontrado 'andean cock-of-the-rock' con entrada en la dbpedia [Andean cock-of-the-rock]'http://dbpedia.org/resource/Andean_cock-of-the-rock'.\n",
 "Hemos encontrado 'humboldt penguin' con entrada en la dbpedia [Humboldt penguin]'http://dbpedia.org/resource/Humboldt_penguin'.\n",
 "Hemos encontrado 'puna plover' con entrada en la dbpedia [Puna plover]'http://dbpedia.org/resource/Puna_plover'.\n",
 "Hemos encontrado 'solitary eagle' con entrada en la dbpedia [Solitary eagle]'http://dbpedia.org/resource/Solitary_eagle'.\n"]

El `entity_recognizer` o `ner` mejora los resultados del apartado anterior pero no la precisión del apartado 3.

## Guardamos el resultado en una Ontología

In [185]:
# !pip install Cython owlready2
from owlready2 import *

#Creamos una ontología
onto = get_ontology("http://avistamiento-aves.org/onto.owl")

class Bird(Thing):
  namespace = onto

from slugify import slugify

found_birds = json.load(open("../data/results/shorebirder_results_3.json", "r"))

for bird in found_birds:
  slugify_name = slugify(bird["name"], separator="_")
  bird_class = types.new_class(slugify_name, (Bird,))

onto.save(file="../owl/avistamiento-aves.xml", format = "rdfxml")

Vamos a poblar nuestra ontologia con los datos obtenidos en el apartado 3, que ha resultado ser la forma más precisa de obtener los pájaros.